In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms

train_dataset = torchvision.datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=100, shuffle=True)

val_dataset = torchvision.datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

val_loader = torch.utils.data.DataLoader(dataset=val_dataset, batch_size=100, shuffle=False)

In [5]:
from lightning_extensions import BaseModule
from models import VAE
from loss import VAELoss

def kl_loss(z_mean, z_log_var):
        return -0.5 * torch.sum(1 + z_log_var - z_mean.pow(2) - z_log_var.exp())
    
def recon_loss(inputs, outputs):
    return F.mse_loss(inputs, outputs, reduction='sum')

class VAEModule(BaseModule):
    def __init__(self):
        model = VAE()
        super().__init__(model)
        self.save_hyperparameters()

    def forward(self, x, y):
        return self.model(x, None, y)

    def step(self, batch, batch_idx, mode = 'train'):
        x, y = batch
        outputs, outputs_masked, z, z_mean, z_log_var = self(x, y)
        loss = {}
        loss['recon_loss_0'] = recon_loss(x, outputs[0])
        loss['kl_loss'] = kl_loss(z_mean, z_log_var)
        loss['loss'] = loss['recon_loss_0'] + loss['kl_loss']
        
        self.log_dict({f"{mode}_{key}": val.item() for key, val in loss.items()}, sync_dist=True, prog_bar=True)
        return loss['loss']

from softadapt import SoftAdapt, NormalizedSoftAdapt, LossWeightedSoftAdapt
class VAEModuleSoftAdapt(BaseModule):
    def __init__(self):
        model = VAE()
        super().__init__(model)
        self.save_hyperparameters()
        self.loss = VAELoss()

    def forward(self, x, y):
        return self.model(x, None, y)

    def step(self, batch, batch_idx, mode = 'train'):
        x, y = batch
        outputs, outputs_masked, z, z_mean, z_log_var = self(x, y)

        loss_dict = self.loss((x, None, y), (outputs, outputs_masked, z, z_mean, z_log_var), mode == 'train')

        self.log_dict({f"{mode}_{key}": val.item() for key, val in loss_dict.items()}, sync_dist=True, prog_bar=True)
        return loss_dict['loss']

In [ ]:
from lightning_extensions import ExtendedTrainer

model = VAEModule()
model_name = "VAE-convolutional"
trainer = ExtendedTrainer(project_name="MTVAEs_SoftAdapt", max_epochs=30, model_name=model_name)
trainer.fit(model, train_loader, val_loader)

In [6]:
from lightning_extensions import ExtendedTrainer

model = VAEModuleSoftAdapt()
model_name = "VAE-convolutional-softadapt"
trainer = ExtendedTrainer(project_name="MTVAEs_SoftAdapt", max_epochs=30, model_name=model_name)
trainer.fit(model, train_loader, val_loader)

/Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:191: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/pyt ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


/Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:630: Checkpoint directory /Home/siv34/edzak2974/projects/MastersThesis/src/checkpoints exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name  | Type    | Params
----------------------------------
0 | model | VAE     | 811 K 
1 | loss  | VAELoss | 0     
----------------------------------
811 K     Trainable params
0         Non-trainable params
811 K     Total params
3.245     Total estimated model params size (MB)


/Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.
/Home/siv34/edzak2974/.conda/envs/pytorch2.1/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=255` in the `DataLoader` to improve performance.


Epoch 29: 100%|██████████| 600/600 [00:10<00:00, 56.25it/s, v_num=gzgs, train_recon_loss_0=3.09e+3, train_kl_loss=762.0, train_loss=2.69e+3, val_recon_loss_0=2.79e+3, val_kl_loss=728.0, val_loss=2.43e+3] 

`Trainer.fit` stopped: `max_epochs=30` reached.


Epoch 29: 100%|██████████| 600/600 [00:10<00:00, 56.24it/s, v_num=gzgs, train_recon_loss_0=3.09e+3, train_kl_loss=762.0, train_loss=2.69e+3, val_recon_loss_0=2.79e+3, val_kl_loss=728.0, val_loss=2.43e+3]


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job
